In [2]:
# Exercises:
# E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?
# E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?
# E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?
# E04: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?
# E05: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?

In [3]:
import torch
import torch.nn.functional as F
import random
with open ('D:/Download/names.txt', 'r') as f:
    namesx = f.read()
az = sorted(set(namesx))
lenaz = len(az)
names = namesx.splitlines()
itos = {i:v for i, v in enumerate(az)}
itos[0] = '.'
stoi = {v:i for i, v in itos.items()}
random.shuffle(names)
train_set = names[:int(0.8*len(names))]
dev_set = names[int(0.8*len(names)):int(0.9*len(names))]
test_set = names [int(0.9*len(names)):]

In [8]:
def build_set (any_set):
    any_xs1, any_xs2, any_ys = [],[],[]
    for name in any_set:
        chs = ['.'] + ['.'] + list(name) + ['.']
        for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
            ix1 = stoi[ch1]
            ix2 = stoi[ch2]
            ix3 = stoi[ch3]
            any_xs1.append(ix1)
            any_xs2.append(ix2)
            any_ys.append(ix3)
    print(len(any_xs1))
    any_xs1 = torch.tensor(any_xs1)
    any_xs2 = torch.tensor(any_xs2)
    any_ys = torch.tensor(any_ys)
    num = any_xs1.shape[0]
    return any_xs1, any_xs2, any_ys, num
    
def lossfw(any_xs1, any_xs2, any_ys, any_num):# ,any_xenc):
    # logits = any_xenc @ W
    logits = W[any_xs1]+W[lenaz+any_xs2]
    # counts = logits.exp()
    # probs = counts/counts.sum(1,keepdim = True)
    # p = probs[torch.arange(any_num), any_ys]
    # loss = -p.log().mean() + reglr*(W**2).mean()
    loss = F.cross_entropy(logits,any_ys)
    return loss
    
def enc(any_xs1, any_xs2):
    any_xenc1 = F.one_hot(any_xs1, num_classes = lenaz).float()
    any_xenc2 = F.one_hot(any_xs2, num_classes = lenaz).float()
    any_xenc = torch.cat((any_xenc1,any_xenc2),1)
    return any_xenc

In [5]:
tr_xs1, tr_xs2, tr_ys, tr_num = build_set (train_set)
reglr = 0
iterk = 501
lr = 10
g = torch.Generator().manual_seed(2147483647)  
W = torch.randn(2*lenaz,lenaz,generator=g,requires_grad=True)
# tr_enc = enc(tr_xs1, tr_xs2)
for k in range (iterk):
    tr_loss = lossfw(tr_xs1, tr_xs2, tr_ys, tr_num)#, tr_enc)
    if k % 100 ==0:
        print(k, tr_loss)
    W.grad = None
    tr_loss.backward()
    with torch.no_grad():
        W -= lr*W.grad
# Bigram loss: 2.4900
# Trigram tr_loss: 2.3632

182579
0 tensor(4.2337, grad_fn=<NllLossBackward0>)
100 tensor(2.4807, grad_fn=<NllLossBackward0>)
200 tensor(2.4085, grad_fn=<NllLossBackward0>)
300 tensor(2.3832, grad_fn=<NllLossBackward0>)
400 tensor(2.3707, grad_fn=<NllLossBackward0>)
500 tensor(2.3632, grad_fn=<NllLossBackward0>)


In [198]:
# reglr = 0; tr_loss = 2.3619
# reglr = 0.0001; tr_loss = 2.3620
# reglr = 0.001; tr_loss = 2.3629
# reglr = 0.01; tr_loss = 2.3708
# reglr = 0.1; tr_loss = 2.4238
# reglr = 1; tr_loss = 2.6324
# reglr = 10; tr_loss = 3.0265

In [6]:
# reglr = 0; d_loss = 2.3668
# reglr = 0.0001; d_loss = 2.3669
# reglr = 0.001; d_loss = 2.3678
# reglr = 0.01; d_loss = 2.3756
# reglr = 0.1; d_loss = 2.4276
# reglr = 1; d_loss = 2.6329
# reglr = 10; d_loss = 3.0252

# therefore I choose reglr=0 with min d_loss

In [268]:
# evaluate dev_set
with torch.no_grad():
    d_xs1, d_xs2, d_ys, d_num = build_set(dev_set)
    # d_enc = enc(d_xs1, d_xs2)
    d_loss = lossfw(d_xs1,d_xs2 ,d_ys,d_num)
d_loss

22827


tensor(2.3668)

In [271]:
# evaluate test_set
with torch.no_grad():
    t_xs1, t_xs2, t_ys, t_num = build_set(test_set)
    # t_enc = enc(t_xs1, t_xs2)
    t_loss = lossfw(t_xs1,t_xs2,t_ys,t_num)
t_loss

22816


tensor(2.3730)

In [270]:
# t_loss = 2.3730

In [9]:
g = torch.Generator().manual_seed(2147483647)
ix1,ix2 = 0,0
for i in range (10):
    sample = []
    while True:
        xenc1 = F.one_hot(torch.tensor([ix1]), num_classes = lenaz).float()
        xenc2 = F.one_hot(torch.tensor([ix2]), num_classes = lenaz).float()
        xenc = torch.cat((xenc1, xenc2),1)
        logits = xenc @ W
        counts = logits.exp()
        probs = counts/counts.sum(1,keepdim = True)
        #print(probs)
        ix1 = ix2
        ix2 = torch.multinomial(probs, 1, replacement = True, generator = g).item()
        sample.append(itos[ix2])
        if ix2==0:
            break
    print(f"{i} {''.join(sample)}")

0 cexze.
1 comakurailaziayh.
2 mellimittain.
3 lusan.
4 ka.
5 da.
6 samiyaubjabhrigotai.
7 kolielliau.
8 na.
9 teda.
